# Homework 4

Solve exercises 10.2, 10.15, 10.19, 11.15

In the next cell, we copy all the functions from the last homework. We will use them for comparison with the new ones.

In [ ]:
function LU(A)
    n, m = size(A) # A is supposed to be a square matrix, so hopefully n and m will be equal.
    
    # We initalize L with zeros and U to be the same as A.
    L = zeros(n,m)
    U = copy(A)
    
    for k in 1:n
        L[k,k] = 1
        for i in (k+1):n
            L[i,k] = U[i,k]/U[k,k]
            U[i,:] = U[i,:] - L[i,k]*U[k,:]
        end
    end
    return L, U
end

function backward_substitute(U,y)
    n, m = size(U)
    r, = size(y)
    @assert n == m == r
    x = zeros(n)
    
    for i in n:-1:1
        tail = 0
        for j in i+1:n
            tail = tail + U[i,j]*x[j]
        end
        x[i] = (y[i] - tail)/U[i,i]
    end
    
    return x
end

function forward_substitute(L,y)
    n, m = size(L)
    r, = size(y)
    @assert n == m == r
    x = zeros(n)
    for i in 1:n
        tail = 0
        for j in 1:i-1
            tail = tail + L[i,j]*x[j]
        end
        x[i] = (y[i] - tail)/L[i,i]
    end
    return x
end

function solve(A,f)
    n, m = size(A)
    r, = size(f)
    @assert n == m == r
    
    # Something goes here
    L, U = LU(A)
    y = forward_substitute(L,f)
    x = backward_substitute(U,y)
    
    return(x)
end

function big_matrix(n)
    A = zeros(n^2,n^2)
    for i in 1:n^2
        A[i,i] = 4
        if mod(i,n)!=0
            A[i+1,i] = -1
            A[i,i+1] = -1
        end
        if i<=n^2-n
            A[i,i+n] = -1
            A[i+n,i] = -1
        end
    end
    return A
end

function big_rhs(n, f)
    y = zeros(n^2)
    for i in 1:n
        y[i] += f(i/(n+1),0.)
        y[n^2-n+i] += f(i/(n+1),1.)
    end
        
    for i in 1:n
        y[n*i] += f(1.,i/(n+1))
        y[n*i-n+1] += f(0.,i/(n+1))
    end
    return y
end

function T(u::Array{<:Real,2})
    n, m = size(u)
    v = similar(u) # This line creates another array with the same dimensions and type as u
    for i in 1:n
        for j in 1:m
            v[i,j] = 4*u[i,j]
            if i>1 v[i,j] -= u[i-1,j] end
            if i<n v[i,j] -= u[i+1,j] end
            if j>1 v[i,j] -= u[i,j-1] end
            if j<m v[i,j] -= u[i,j+1] end
        end
    end
    return v
end

function matrix_rhs(n::Integer, f::Function)
    b = zeros(n,n)
    for i in 1:n
        b[1,i] += f(0.,i/(n+1))
        b[n,i] += f(1.,i/(n+1))
        b[i,1] += f(i/(n+1),0.)
        b[i,n] += f(i/(n+1),1.)
    end
    return b
end  

# The following function is supposed to be equivalent as doing forward substitution with the lower-triangular 
# part of the matrix. Since the matrix is sparce, it is much more efficient to implement it as a function than 
# to write extract the lower-triangular part of A and called forward_substitute from above.

function forward_substitute_lower_diagonal_part(v::Array{<:Real,2})
    n, m = size(v)
    u = zeros(n,m)
    for i in 1:n
        for j in 1:m
            s = v[i,j]
            if i>1 s += u[i-1,j] end
            if j>1 s += u[i,j-1] end
            u[i,j] = 0.25 * s
        end
    end
    return u
end

# The following function corresponds to applying the upper triangular part of A to u.
function upper_triangular_part(u::Array{<:Real,2})
    n, m = size(u)
    v = zeros(n,m)
    for i in 1:n
        for j in 1:m
            if i<n v[i,j] -= u[i+1,j] end
            if j<m v[i,j] -= u[i,j+1] end
        end
    end
    return v
end

function gauss_seidel(u::Array{<:Real,2}, b::Array{<:Real,2}, iterations::Integer)
    for i in 1:iterations
        u = forward_substitute_lower_diagonal_part(b - upper_triangular_part(u))
    end
    return u
end    

function inner_product(u::Array{<:Real,2}, v::Array{<:Real,2})
    s = 0
    n,m = size(u)
    for i in 1:n
        for j in 1:m
            s += u[i,j] * v[i,j]
        end
    end
    return s
end

function gradient_descent(u::Array{<:Real,2}, b::Array{<:Real,2}, iterations::Integer)
    for i in 1:iterations
        r = T(u) - b
        a = inner_product(r,r) / inner_product(T(r),r)
        u = u - a*r
    end
    return u
end

Now you are supposed to implement the conjugate gradients method.

In [ ]:
function conjugate_gradients(u::Array{<:Real,2}, b::Array{<:Real,2}, iterations::Integer)
    # ...
end

Let us test how long these algorithms take in practice. So, we will time their execution time with a 40x40 mesh.

In [ ]:
n = 40
f(x,y) = 1 - sin(pi*x)
partition = 1/(n+1) : 1/(n+1) : 1-1/(n+1)

# We use the following function to roughly measure the size of vectors or matrices.
function supnorm(u)
    s = 0
    for entry in u
        s = max(abs(entry),s)
    end
    return s
end

# Using Gaussian elimination
y = big_rhs(n,f)
A = big_matrix(n)
@time x = solve(A,y)
println("Accuracy of Gaussian elimination: ", supnorm(A*x-y) / n^2)

# Using Gauss-Seidel iteration
# We can test different number of iterations

iterations = div(n^2,10)
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = gauss_seidel(u,b,iterations)
println("Accuracy of Gauss Seidel with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

iterations = n^2
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = gauss_seidel(u,b,iterations)
println("Accuracy of Gauss Seidel with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

iterations = 10*n^2
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = gauss_seidel(u,b,iterations)
println("Accuracy of Gauss Seidel with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

# Using Gradient descent
# We can test different number of iterations

iterations = n^2
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = gradient_descent(u,b,iterations)
println("Accuracy of Gradient descent with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

# Using Conjugate radients
# We can test different number of iterations

iterations = div(n^2,10)
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = conjugate_gradients(u,b,iterations)
println("Accuracy of conjugate gradients with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

iterations = n^2
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = conjugate_gradients(u,b,iterations)
println("Accuracy of conjugate gradients with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

Conjugate gradients wins by a large margin. In this example, we are getting the exact solution in even fewer iterations than expected. I don't have a good explanation of why, and we shouldn't expect that in general.

We can even solve the problem with a massive mesh of 200x200. It takes less than a minute to get the exact solution in my (old) office computer. Good luck doing that with any of the other methods.

In [ ]:
n = 200
f(x,y) = 1 - sin(pi*x)
partition = 1/(n+1) : 1/(n+1) : 1-1/(n+1)
iterations = n^2
b = matrix_rhs(n,f)
u = zeros(n,n)
@time u = conjugate_gradients(u,b,iterations)
println("Accuracy of conjugate gradients with ", iterations, " iterations: ", supnorm(T(u)-b)/n^2)

## Final fun movie

Let us make a movie to compare the speed of gradient descent with conjugate gradients.

In [ ]:
using Plots
iterations = 5
n = 40
partition = 1/(n+1) : 1/(n+1) : 1-1/(n+1)

let b = matrix_rhs(n,f), u = zeros(n,n), v = zeros(n,n)
    @gif for i in 1:300
        u = gradient_descent(u,b,iterations)
        v = conjugate_gradients(v,b,iterations)
        sgd1 = surface(partition,partition,u, title=string(i*iterations)*" iterations")
        sgd2 = surface(partition,partition,v, title=string(i*iterations)*" iterations")
        sg = plot(sgd1,sgd2,layout=2, size=(800,300))
    end
end